In [ ]:
# -*- coding: Paper1 -*-
"""
This code belongs to the paper:
-- An Iterative Gradient Descent-Based Reinforcement Learning Policy 
   for Active Control of Structural Vibrations, Computers & Structures.
-- Jagajyoti Panda, Mudit Chopra, Vasant Matsagar, Souvik Chakraborty, IIT Delhi.
   
This code is to present the robust performance of trained RL controllers in P and PI feedback.
-- Case study II: 8-story benchmark building (section 5.2.2)
"""
from gym import Env
from gym.spaces import Discrete, Box
import numpy as np
import random
import os
from numpy import abs as Abs
from numpy import sign as sign
from numpy import sqrt,pi,array
from scipy import signal
 
 
import matplotlib
import matplotlib.pyplot as plt

from control.matlab import *  # MATLAB-like functions
import control as ct
 
import tensorflow as tf
from gym import Env
from gym.spaces import Discrete, Box
 
 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

# Ground motion from SAC database
GMdt = np.loadtxt('GMdt.txt', unpack = True)
print(len(GMdt))

# Trained Proportional Controller (8storey)
BKt = np.load('BKt_P8.npy')
Ad = np.load('Ad_P8.npy')
Bd = np.load('Bd_P8.npy')
Cd1 = np.load('Cd1_P8.npy')
Cd = np.load('Cd_P8.npy')
Dd = np.load('Dd_P8.npy')
Ed = np.load('Ed_P8.npy')
Kpo = np.load('Kpo_P8.npy')
print(BKt)

# Trained PI Controller (8storey)
BKti = np.load('BKt_PI8.npy')
FI = np.load('FI_PI8.npy')
gI = np.load('gI_PI8.npy')
Cd1i = np.load('Cd1_PI8.npy')
Cdi = np.load('Cdi_PI8.npy')
Ddi = np.load('Ddi_PI8.npy')
BIrd = np.load('Ed_PI8.npy')
Kpoi = np.load('Kpo_PI8.npy')
print(BKti)

no = np.linalg.norm(Bd@Kpo)
noi = np.linalg.norm(gI@Kpoi)
print(no, noi)

npg = np.linalg.norm(BKt)
npgi = np.linalg.norm(BKti)
print(npg, npgi)

# Proposed controller (inv(B)@BKt)
Kpt = (np.linalg.pinv(Bd)@BKt)
Kpti = (np.linalg.pinv(gI)@BKti)
print(Kpt.shape, Kpti.shape)

n = 8
r =3
Qp = np.identity(2*n)
Qpi = np.identity(2*n+r)
print(Qpi.shape)

# Trained results
xr = np.zeros([4100, 2*n, len(GMdt)])  # State
yr = np.zeros([4100, len(Cd1), len(GMdt)])  # output
xri = np.zeros([4100, 2*n+r, len(GMdt)])  # State
yri = np.zeros([4100, len(Cd1i), len(GMdt)])  # output
# Optimal results
xo = np.zeros([4100, 2*n, len(GMdt)])
xoi = np.zeros([4100, 2*n+r, len(GMdt)])
xu= np.zeros([4100, 2*n, len(GMdt)])
(i,j) = Cd.shape
yu = np.zeros([4100, i, len(GMdt)])
yo = np.zeros([4100, i, len(GMdt)])
yoi = np.zeros([4100, i, len(GMdt)])

# Initialize performance indices
J1r = np.zeros([len(GMdt), n])
J2r = np.zeros([len(GMdt), n])
J3r = np.zeros([len(GMdt), n])
J4r = np.zeros([len(GMdt), 1])
J5r = np.zeros([len(GMdt), 3])
J1o = np.zeros([len(GMdt), n])
J2o = np.zeros([len(GMdt), n])
J3o = np.zeros([len(GMdt), n])
J4o = np.zeros([len(GMdt), 1])
J5o = np.zeros([len(GMdt), 3])

J1ri = np.zeros([len(GMdt), n])
J2ri = np.zeros([len(GMdt), n])
J3ri = np.zeros([len(GMdt), n])
J4ri = np.zeros([len(GMdt), 1])
J5ri = np.zeros([len(GMdt), 3])
J1oi = np.zeros([len(GMdt), n])
J2oi = np.zeros([len(GMdt), n])
J3oi = np.zeros([len(GMdt), n])
J4oi = np.zeros([len(GMdt), 1])
J5oi = np.zeros([len(GMdt), 3])

Euct = np.zeros([4100, 1, len(GMdt)]) 
Eot = np.zeros([4100, 1, len(GMdt)])  
Et = np.zeros([4100, 1, len(GMdt)]) 
Fot = np.zeros([4100, r, len(GMdt)])  
Ft = np.zeros([4100, r, len(GMdt)]) 

Eoti = np.zeros([4100, 1, len(GMdt)])  
Eti = np.zeros([4100, 1, len(GMdt)]) 
Foti = np.zeros([4100, r, len(GMdt)])  
Fti = np.zeros([4100, r, len(GMdt)]) 

# Ground motion input
for ijk in range(49,len(GMdt)+1):
    if ijk < 10:
        filename="la0" + str(ijk) + ".dat"
        print(filename)
    else:
        filename="la" + str(ijk) + ".dat"
        print(filename)
     
    data = np.loadtxt(filename, unpack = True)

    xacc=np.full([len(data)*len(data[0])],None)
    for x in range(len(data[0])):
        for y in range(len(data)):
            if x == 0:
                xacc[y] = data[y, x]
            else:
                xacc[y+(x*6)] = data[y, x]
    xacc = xacc/100
    print(xacc)
    # 
    # Time series for ground motion
    dt = GMdt[ijk-1]
    tend = dt*len(xacc)
    tim = np.arange(0, tend, dt)
    plt.figure()
    plt.plot(tim,xacc)
    plt.show(block=False)
    for tkk in range(0,len(tim)-1):
        time_current = tim[tkk]
        h = xacc[tkk] 
        xn = np.array([xr[tkk,:,ijk-1]]).T
        xn1 = (Ad @ xn) + (BKt @ xn) + (Ed * h)
        #     print(xn1)
        xr[tkk+1,:,ijk-1] = xn1.T
        En2 = 0.5*((xn1.T @ Qp)@ (xn1))
        Et[tkk+1,:,ijk-1] = En2
        Fn2 = Kpt @ xn1
        Ft[tkk+1,:,ijk-1] = Fn2.T
        yn1 = (Cd1-BKt) @ xn1
        yr[tkk+1,:,ijk-1] = yn1.T
        del xn, xn1, En2, Fn2, yn1
        # print(sinewave)
        
        y = (Cd @ np.array([xu[tkk,:,ijk-1]]).T)
        yu[tkk,:,ijk-1] = y.T
        x = (Ad @ np.array([xu[tkk,:,ijk-1]]).T) + (Ed * h)
        xu[tkk+1,:,ijk-1] = x.T
        En1 = 0.5*((x.T @ Qp)@ (x))
        Euct[tkk+1,:,ijk-1] = En1
        del y, x, En1
        
        y1 = (Cd @ np.array([xo[tkk,:,ijk-1]]).T) + (Dd @ ((-1*Kpo) @ np.array([xo[tkk,:,ijk-1]]).T))
        yo[tkk,:,ijk-1] = y1.T
        x1 = (Ad @ np.array([xo[tkk,:,ijk-1]]).T) + (Bd @ ((-1*Kpo) @ np.array([xo[tkk,:,ijk-1]]).T)) + (Ed * h)
        xo[tkk+1,:,ijk-1] = x1.T
        En0 = 0.5*((x1.T @ Qp)@ (x1))
        Eot[tkk+1,:,ijk-1] = En0
        Fn1 = Kpo @ x1
        Fot[tkk+1,:,ijk-1] = Fn1.T
        del y1, x1, En0, Fn1 
        
        ## PI controller
        xn = np.array([xri[tkk,:,ijk-1]]).T
        xn1 = (FI @ xn) + (BKti @ xn) + (BIrd * h)
#         print(xn1)
        xri[tkk+1,:,ijk-1] = xn1.T
        En2 = 0.5*((xn1.T @ Qpi)@ (xn1))
        Eti[tkk+1,:,ijk-1] = En2
        Fn2 = Kpti @ xn1
        Fti[tkk+1,:,ijk-1] = Fn2.T
        yn1 = (Cd1i-BKti) @ xn1
        yri[tkk+1,:,ijk-1] = yn1.T
        del xn, xn1, En2, Fn2, yn1
        
        y1 = (Cdi @ np.array([xoi[tkk,:,ijk-1]]).T) + (Ddi @ ((-1*Kpoi) @ np.array([xoi[tkk,:,ijk-1]]).T))
        yoi[tkk,:,ijk-1] = y1.T
        x1 = (FI @ np.array([xoi[tkk,:,ijk-1]]).T) + (gI @ ((-1*Kpoi) @ np.array([xoi[tkk,:,ijk-1]]).T)) + (BIrd * h)
        xoi[tkk+1,:,ijk-1] = x1.T
#         print(Qpi)
        En0 = 0.5*((x1.T @ Qpi)@ (x1))
        Eoti[tkk+1,:,ijk-1] = En0
        Fn1 = Kpoi @ x1
        Foti[tkk+1,:,ijk-1] = Fn1.T
        del y1, x1, En0, Fn1 
        
# Performance indices   
    for i in range(0,n):
        J1r[ijk-1, i] = max(xr[:,i,ijk-1], key=abs)/max(xu[:,i,ijk-1], key=abs)
        J1o[ijk-1, i] = max(xo[:,i,ijk-1], key=abs)/max(xu[:,i,ijk-1], key=abs)
        J2r[ijk-1, i] = max(xr[:,n+i,ijk-1], key=abs)/max(xu[:,n+i,ijk-1], key=abs)
        J2o[ijk-1, i] = max(xo[:,n+i,ijk-1], key=abs)/max(xu[:,n+i,ijk-1], key=abs)
        J3r[ijk-1, i] = max(yr[:,i,ijk-1], key=abs)/max(yu[:,i,ijk-1], key=abs)
        J3o[ijk-1, i] = max(yo[:,i,ijk-1], key=abs)/max(yu[:,i,ijk-1], key=abs)
        
        J1ri[ijk-1, i] = max(xri[:,i,ijk-1], key=abs)/max(xu[:,i,ijk-1], key=abs)
        J1oi[ijk-1, i] = max(xoi[:,i,ijk-1], key=abs)/max(xu[:,i,ijk-1], key=abs)
        J2ri[ijk-1, i] = max(xri[:,n+i,ijk-1], key=abs)/max(xu[:,n+i,ijk-1], key=abs)
        J2oi[ijk-1, i] = max(xoi[:,n+i,ijk-1], key=abs)/max(xu[:,n+i,ijk-1], key=abs)
        J3ri[ijk-1, i] = max(yri[:,i,ijk-1], key=abs)/max(yu[:,i,ijk-1], key=abs)
        J3oi[ijk-1, i] = max(yoi[:,i,ijk-1], key=abs)/max(yu[:,i,ijk-1], key=abs)
        
    J4r[ijk-1, 0] = max(Et[:,:,ijk-1], key=abs)/max(Euct[:,:,ijk-1], key=abs)
    J4o[ijk-1, 0] = max(Eot[:,:,ijk-1], key=abs)/max(Euct[:,:,ijk-1], key=abs)
    J4r[ijk-1, 0] = max(Et[:,:,ijk-1], key=abs)/max(Euct[:,:,ijk-1], key=abs)
    J4o[ijk-1, 0] = max(Eot[:,:,ijk-1], key=abs)/max(Euct[:,:,ijk-1], key=abs)
    for i in range(0,r):
        J5r[ijk-1, i] = (max(Ft[:,i,ijk-1], key=abs)*(npg*10/no))/(345600*10*8)
        J5o[ijk-1, i] = max(Fot[:,i,ijk-1], key=abs)/(345600*10*8)
        J5ri[ijk-1, i] = (max(Fti[:,i,ijk-1], key=abs)*(npg*10/no))/(345600*10*8)
        J5oi[ijk-1, i] = max(Foti[:,i,ijk-1], key=abs)/(345600*10*8)

# P controller - Performance index J1 (Section 5.2.2, Figure 11)
Floor = [1, 2, 3, 4, 5, 6, 7, 8]
print("J1 - Proposed P:\n", "    Floor1      Floor2      Floor3     Floor4      Floor5       Floor6 \n     Floor7      Floor8 \n", J1r)
PJ1 = np.zeros([len(GMdt), 1]) 
for ijk in range(0,len(GMdt)):
    PJ1[ijk,:] = max(np.absolute(J1r[ijk,:]))
PJ1[46:56] = PJ1[48:58]


#  Estimation of mean and standard deviation
import numpy as np
import scipy.stats as st
  

# define sample data (hazard levels: 10% in 50 years, 2% in 50 years, 50% in 50 years)
gfg_data10 = PJ1[0:20,0]
gfg_data2 = PJ1[20:40,0]
gfg_data50 = PJ1[40:57,0]  
# create 95% confidence interval (hazard level - 10% in 50 years)
print(np.mean(gfg_data10))
st.t.interval(alpha=0.95, df=len(gfg_data10)-1,
              loc=np.mean(gfg_data10),
              scale=st.sem(gfg_data10))


# create 95% confidence interval (hazard level - 2% in 50 years)
print(np.mean(gfg_data2))
st.t.interval(alpha=0.95, df=len(gfg_data2)-1,
              loc=np.mean(gfg_data2),
              scale=st.sem(gfg_data2))


# create 95% confidence interval (hazard level - 50% in 50 years)
print(np.mean(gfg_data50))
st.t.interval(alpha=0.95, df=len(gfg_data50)-1,
              loc=np.mean(gfg_data50),
              scale=st.sem(gfg_data50))

# PI controller - Performance index J1 (Section 5.2.2, Figure 11)
Floor = [1, 2, 3, 4, 5, 6, 7, 8]
print("J1 - Proposed PI:\n", "    Floor1      Floor2      Floor3     Floor4      Floor5       Floor6 \n     Floor7      Floor8 \n", J1ri)
PJ1i = np.zeros([len(GMdt), 1]) 
for ijk in range(0,len(GMdt)):
    PJ1i[ijk,:] = max(np.absolute(J1ri[ijk,:]))
PJ1i[46:56] = PJ1i[48:58]


# define sample data
gfg_data10 = PJ1i[0:20,0]
gfg_data2 = PJ1i[20:40,0]
gfg_data50 = PJ1i[40:57,0]  
# create 95% confidence interval with mean (10% in 50 years)
print(np.mean(gfg_data10))
st.t.interval(alpha=0.95, df=len(gfg_data10)-1,
              loc=np.mean(gfg_data10),
              scale=st.sem(gfg_data10))


# create 95% confidence interval with mean (2% in 50 years)
print(np.mean(gfg_data2))
st.t.interval(alpha=0.95, df=len(gfg_data2)-1,
              loc=np.mean(gfg_data2),
              scale=st.sem(gfg_data2))

# create 95% confidence interval with mean (50% in 50 years)
print(np.mean(gfg_data50))
st.t.interval(alpha=0.95, df=len(gfg_data50)-1,
              loc=np.mean(gfg_data50),
              scale=st.sem(gfg_data50))